In [ ]:
import csv
import os
import pandas as pd
import numpy as np
import requests
import datetime
from datetime import datetime
import requests
import zipfile
import io
import urllib
from sqlalchemy import create_engine
#from sqlalchemy.ext.automap import automap_base
import psycopg2
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.schema import Sequence

In [ ]:
# uploads the individual contributor datafile directly from the FEC website. 
# Opens Zipped archive and opens the text file.

r = requests.get("https://www.fec.gov/files/bulk-downloads/2020/indiv20.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
file=z.open('itcont.txt')

In [ ]:
columns_dict={'CMTE_ID':'str',
             'AMNDT_IND':'str',
             'RPT_TP':'str',
             'TRANSACTION_PGI':'str',
             'IMAGE_NUM':'str',
             'TRANSACTION_TP':'str',
             'ENTITY_TP':'str',
             'NAME':'str',
             'CITY':'str',
             'STATE':'str',
             'ZIP_CODE':'str',
             'EMPLOYER':'str',
             'OCCUPATION':'str',
             'TRANSACTION_DT':'str',
             'TRANSACTION_AMT':'str',
             'OTHER_ID':'str',
             'TRAN_ID':'str',
             'FILE_NUM':'str',
             'MEMO_CD':'str',
             'MEMO_TEXT':'str',
             "SUB_ID":"str"
            }

columns = list(columns_dict.keys())

In [ ]:
#Converts opened text file into a Pandas Dataframe

individual_cont_staging = pd.read_csv(file, sep='|', index_col=False, names=columns, dtype=columns_dict,
                              parse_dates=['TRANSACTION_DT'])

individual_cont_staging.head()

In [ ]:
# identify the shape of the dataframe.  It's pretty BIG!

individual_cont_staging.shape

In [ ]:
#In effort to convert the date text to date-time field, identified one record that would not convert, becuase the data
#is not in correct format.   Removed that record from dataframe.

# individual_cont_staging.set_index('TRANSACTION_DT')
# error_table = individual_cont_staging.loc['SAN DIEGO']
# error_table

weird = individual_cont_staging['TRANSACTION_DT']=='SAN DIEGO'
weird_df=individual_cont_staging[weird].index
individual_cont1=individual_cont_staging.drop(weird_df)


In [ ]:
#Convert the "TRANSACTION_DT" to date-time field

individual_cont1['TRANSACTION_DT'] = pd.to_datetime(individual_cont1['TRANSACTION_DT'], format="%m%d%Y")

individual_cont1.head()

In [ ]:
# identify the shape of the dataframe.  It's pretty BIG!

individual_cont1.shape

In [ ]:
#Change dtypes of columns that should be numeric to floats.

individual_cont1['TRANSACTION_AMT'] = individual_cont1['TRANSACTION_AMT'].astype(float)

individual_cont1['FILE_NUM'] = individual_cont1['FILE_NUM'].astype(float)

individual_cont1['TRANSACTION_AMT'] = individual_cont1['TRANSACTION_AMT'].astype(float)

individual_cont1['SUB_ID'] = individual_cont1['SUB_ID'].astype(float)

#Check dtype of all the columns.

individual_cont1.dtypes

In [ ]:
individual_cont1.head()

In [ ]:
#Postgres username, password, and database name
ipaddress = 'localhost'
port = '5432'
username = 'postgres'
password = 'password' 
dbname = 'FEC_Data'
# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'


In [ ]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

# Base = automap_base()
Base = declarative_base()
engine = create_engine(postgres_str)
#Base.prepare(engine, reflect=True)

class Individuals(Base):
    __tablename__ = 'individual_cont'
    aid = Column(Integer, primary_key=True, autoincrement=True)
    CMTE_ID = Column(String(9), nullable=False)
    AMNDT_IND = Column(String(1))
    RPT_TP = Column(String(3))
    TRANSACTION_PGI=Column(String(5))
    IMAGE_NUM = Column(String(18))
    TRANSACTION_TP=Column(String(3))
    ENTITY_TP=Column(String(3))
    NAME = Column(String(200))
    CITY = Column(String(30))
    STATE = Column(String(2))
    ZIP_CODE = Column(String(9))
    EMPLOYER = Column(String)
    OCCUPATION = Column(String)
    TRANSACTION_DT = Column(DateTime)
    TRANSACTION_AMT = Column(Float(14))
    OTHER_ID = Column(String(9))
    TRAN_ID = Column(String(32))
    FILE_NUM = Column(Float(22))
    MEMO_CD = Column(String(1))
    MEMO_TEXT = Column(String(100))
    SUB_ID = Column(Float(19), nullable=False)
#     __table_args__ = (
#         PrimaryKeyConstraint('SUB_ID','TRAN_ID','FILE_NUM','CMTE_ID'),{})
    
Base.metadata.create_all(engine)

individual_cont1.to_sql('individual_cont', engine, if_exists='append', index=False)